In [ ]:
# default_exp models.core
# default_cls_lvl 3

# Models
> Pytorch Models for Sequential Data

In [ ]:
#export
from seqdata.core import *
from fastai.basics import *
from fastai.callback.progress import *
from torch.nn.utils import weight_norm
from torch.nn import Parameter

In [ ]:
seq = DataBlock(blocks=(SequenceBlock.from_hdf(['current','voltage'],TensorSequencesInput,clm_shift=[0,-1]),
                        SequenceBlock.from_hdf(['voltage'],TensorSequencesOutput,clm_shift=[-1])),
                 get_items=CreateDict([DfHDFCreateWindows(win_sz=1000+1,stp_sz=1000,clm='current')]),
                 splitter=ApplyToDict(ParentSplitter()))
db = seq.dataloaders(get_hdf_files('test_data/'))

## Batchnorm

In [ ]:
#export
class BatchNorm_1D_Stateful(nn.Module):
    '''Batchnorm for stateful models. Stores batch statistics for for every timestep seperately to mitigate transient effects.'''
    __constants__ = ['track_running_stats', 'momentum', 'eps', 'weight', 'bias',
                     'running_mean', 'running_var', 'num_batches_tracked']

    def __init__(self, hidden_size, seq_len=None,stateful=False,batch_first=True, eps=1e-7, momentum=0.1, affine=True,
                 track_running_stats=True):#num_features
        super().__init__()
        channel_d=hidden_size
        self.seq_len = seq_len
        self.stateful = stateful
        self.batch_first = batch_first
        self.eps = eps
        self.momentum = momentum
        self.affine = affine
        self.track_running_stats = track_running_stats
        self.axes=(1,)
        if self.affine:
            self.weight = Parameter(torch.Tensor(channel_d))
            self.bias = Parameter(torch.Tensor(channel_d))
            self.register_parameter('weight', self.weight)
            self.register_parameter('bias', self.bias)
        else:
            self.register_parameter('weight', None)
            self.register_parameter('bias', None)
        if self.track_running_stats:
            if seq_len is not None:
                self.register_buffer('running_mean', torch.zeros(seq_len,channel_d))
                self.register_buffer('running_var', torch.ones(seq_len,channel_d))
            self.register_buffer('num_batches_tracked', torch.tensor(0, dtype=torch.long))
        else:
            self.register_parameter('running_mean', None)
            self.register_parameter('running_var', None)
            self.register_parameter('num_batches_tracked', None)
        self.reset_parameters()
        self.reset_state()
        
    def reset_state(self):
        self.seq_idx = 0

    def reset_parameters(self):
        if self.track_running_stats and self.seq_len is not None:
            self.running_mean.zero_()
            self.running_var.fill_(1)
            self.num_batches_tracked.zero_()
        if self.affine:
            self.weight.data.fill_(1.0)
            self.bias.data.fill_(0.0)
            
    def forward(self, input, BN_start=None):
        if input.dim() != 3:
            raise ValueError('expected 3D input (got {}D input)'.format(input.dim()))
        if self.batch_first: input = input.transpose(0,1)
            
        input_t,n_batch,hidden_size =input.size()
        
        if self.track_running_stats and self.seq_len is None:
            self.seq_len = input_t
            self.register_buffer('running_mean', torch.zeros((input_t,hidden_size),device=input.device))
            self.register_buffer('running_var', torch.ones((input_t,hidden_size),device=input.device))
            
        if BN_start is None:
            if self.stateful:
                BN_start = self.seq_idx
            else:
                BN_start = 0

        exponential_average_factor = 0.0
        if self.training and self.track_running_stats:
            if self.num_batches_tracked is not None:
                self.num_batches_tracked += 1
                if self.momentum is None:  # use cumulative moving average
                    exponential_average_factor = 1.0 / float(self.num_batches_tracked)
                else:  # use exponential moving average
                    exponential_average_factor = self.momentum

        BN_stop = BN_start+input_t
        self.seq_idx = BN_stop #new starting point for next forward call 
        
        
        if self.training:
            mean = input.mean(1)
            var = input.var(1, unbiased=False)# use biased var in train
            
            if self.seq_len-BN_start > 0: #frame has to be in statistics window for updates
                with torch.no_grad():
                    self.running_mean[BN_start:BN_stop] = exponential_average_factor * mean[:self.seq_len-BN_start] + (1 - exponential_average_factor) * self.running_mean[BN_start:BN_stop]
                    self.running_var[BN_start:BN_stop] = exponential_average_factor * var[:self.seq_len-BN_start] * n_batch / (n_batch - 1)  + (1 - exponential_average_factor) * self.running_var[BN_start:BN_stop] # update running_var with unbiased var
        else:        
            mean = self.running_mean[BN_start:BN_stop]
            var = self.running_var[BN_start:BN_stop]
            
            #if elements outside of the statistics are requested, append the last element repeatedly
#             import pdb;pdb.set_trace()
            if BN_stop >= self.seq_len:
                cat_len = input_t - max(self.seq_len-BN_start,0)  # min(BN_stop-self.seq_len,self.seq_len) 
                mean = torch.cat((mean,self.running_mean[-1:].repeat(cat_len,1)))
                var = torch.cat((var,self.running_var[-1:].repeat(cat_len,1)))

        output = (input - mean[:, None, :]) / (torch.sqrt(var[:, None, :] + self.eps))
        if self.affine:
            output = output * self.weight[None, None, :] + self.bias[None, None, :]#[None, :, None, None]
            

        if self.batch_first: output = output.transpose(0,1)
            
        return output

## Linear

In [ ]:
#export
class SeqLinear(nn.Module):
    
    def __init__(self,input_size,output_size,hidden_size=100,hidden_layer=1,act=Mish,batch_first=True):
        super().__init__()
        self.batch_first = batch_first
        def conv_act(inp,out): return nn.Sequential(nn.Conv1d(inp,out,1),act())
        
        if hidden_layer < 1:
            self.lin = nn.Conv1d(input_size,output_size,1)
        else:
            self.lin = nn.Sequential(conv_act(input_size,hidden_size),
                                     *[conv_act(hidden_size,hidden_size) for _ in range(hidden_layer-1)],
                                    nn.Conv1d(hidden_size,output_size,1))
            
    def forward(self, x):
        if not self.batch_first: x = x.transpose(0,1)
        out = self.lin(x.transpose(1,2)).transpose(1,2)
        
        if not self.batch_first: out = out.transpose(0,1)
        return out

## RNNs

### Basic RNN

In [ ]:
# #export
# from sru import SRU as SRULayer
# class SRU(nn.Module):
    
#     @delegates(SRULayer, keep=True)
#     def __init__(self, input_size, hidden_size,num_layers = 1, batch_first=True, **kwargs):
#         super().__init__()        
#         self.batch_first = batch_first
#         self.rnn = SRULayer(input_size,hidden_size,num_layers,**kwargs)

    
#     def forward(self, inp, hid=None):
# #         import pdb;pdb.set_trace()
#         if self.batch_first: inp = inp.transpose(0,1)
#         out,hid = self.rnn(inp,hid)
#         if self.batch_first: out = out.transpose(0,1)
#         return out, hid

In [ ]:
#export
from fastai.text.models.awdlstm import *
from seqdata.models.indrnn import IndRNN
class RNN(nn.Module):
    "inspired by https://arxiv.org/abs/1708.02182"

    def __init__(self, input_size,hidden_size, num_layers, 
                 hidden_p=0.0, input_p=0.0, weight_p=0.0,
                 rnn_type='gru',ret_full_hidden=False,stateful=False,
                 normalization='',**kwargs):
        super().__init__()
        store_attr('ret_full_hidden,num_layers,rnn_type,hidden_size,stateful,input_p,normalization')
        self.bs = 1
        
        self.rnns = nn.ModuleList([self._one_rnn(input_size if l == 0 else hidden_size,
                                                 hidden_size,weight_p,rnn_type,**kwargs) for l in range(num_layers)])
        
        self.input_dp = RNNDropout(input_p)
        self.hidden_dps = nn.ModuleList([RNNDropout(hidden_p) for l in range(num_layers)])
        
        if normalization == '':
            self.norm_layers = [None]*num_layers
        elif normalization == 'layernorm':
            self.norm_layers = nn.ModuleList([nn.LayerNorm(hidden_size,elementwise_affine=False) for l in range(num_layers)])
        elif normalization == 'batchnorm':
            self.norm_layers = nn.ModuleList([(BatchNorm_1D_Stateful(hidden_size,stateful=stateful,batch_first=True,affine=False)) for i in range(num_layers)])  
        else:
            raise ValueError('Wrong Value for normalization')
        self.reset_state()

    def forward(self, inp, h_init=None):
        bs,seq_len,_ = inp.shape
        if h_init is None and self.stateful: h_init = self._get_hidden(bs)
                
        r_input = self.input_dp(inp) if self.input_p > 0 else inp
        full_hid,new_hidden = [],[]
#         import pdb; pdb.set_trace()
        for l, (rnn,hid_dp,nrm) in enumerate(zip(self.rnns,self.hidden_dps,self.norm_layers)):
            r_output, h = rnn(r_input,h_init[l] if h_init is not None else None)
            
            if self.normalization != '':
                r_output = nrm(r_output)
                
            if l != self.num_layers - 1: 
                r_output = hid_dp(r_output)
            
            full_hid.append(r_output)
            new_hidden.append(h)
            r_input = r_output
        
        self.hidden =  to_detach(new_hidden, cpu=False, gather=False)
        self.bs = bs
        output = r_output if not self.ret_full_hidden else torch.stack(full_hid, 0)
        
        return output, new_hidden

    def _get_hidden(self,bs):
        '''retrieve internal hidden state, check if model device has changed'''
        if self.hidden is None: return None
        if bs!=self.bs: return None
        if self.hidden[0][0].device != one_param(self).device: return None
#         import pdb; pdb.set_trace()
        return self.hidden
    
    def _one_rnn(self, n_in, n_out, weight_p, rnn_type,**kwargs):
        "Return one of the inner rnn"
        if rnn_type == 'gru':
            rnn = nn.GRU(n_in, n_out,1,batch_first=True,**kwargs)
#             rnn = WeightDropout(rnn,weight_p)
        elif rnn_type == 'lstm':
            rnn = nn.LSTM(n_in, n_out,1,batch_first=True,**kwargs)
#             rnn = WeightDropout(rnn,weight_p)
        elif rnn_type == 'sru':
            rnn = SRU(n_in, n_out,1,batch_first=True,**kwargs)
#             rnn = WeightDropout(rnn,weight_p)
        elif rnn_type == 'rnn':
            rnn = nn.RNN(n_in, n_out,1,batch_first=True,**kwargs)
#             rnn = WeightDropout(rnn,weight_p)
        elif rnn_type == 'qrnn':
            rnn = QRNN(n_in, n_out,1,batch_first=True,**kwargs)
#             rnn.layers[0].linear = WeightDropout(rnn.layers[0].linear,weight_p,layer_names='weight')
        elif rnn_type == 'indrnn':
            rnn = IndRNN(n_in, n_out,1,batch_first=True,**kwargs)
        else:
            raise Exception
        return rnn
    
    def reset_state(self):
        self.hidden = None

In [ ]:
#export
class Sequential_RNN(RNN):
    '''RNN Variant for Sequential Modules'''
    def forward(self, inp, h_init=None):
        return super().forward(inp, h_init)[0]

In [ ]:
# #export
# class SimpleRNN(nn.Module):
    
#     @delegates(RNN, keep=True)
#     def __init__(self,input_size,output_size,num_layers=1,hidden_size=100,lrn_init=False,linear_layers=1,**kwargs):
#         super().__init__()
#         self.rnn = RNN(input_size=input_size,hidden_size=hidden_size,num_layers=num_layers,**kwargs)
#         self.final = SeqLinear(hidden_size,output_size,hidden_size=hidden_size,hidden_layer=linear_layers,act=nn.LeakyReLU)
        
#         self.init_p = (nn.ParameterList([nn.Parameter(torch.randn(size=(1,1,hidden_size)),requires_grad=True) 
#                         for f in range(num_layers)]) if lrn_init else None)
#         self.reset_flag = False
#     def forward(self, x):
#         if self.init_p is None or not self.reset_flag:
#             h_init = None
#         else:
# #             import pdb; pdb.set_trace()
#             h_init = [p.repeat((1,x.shape[0],1)) for p in self.init_p]
#             self.reset_flag = False
        
#         out,_ = self.rnn(x,h_init)
#         out = self.final(out)
#         return out
#     def reset_state(self):
#         self.reset_flag = True

In [ ]:
#export
class SimpleRNN(nn.Module):
    
    @delegates(RNN, keep=True)
    def __init__(self,input_size,output_size,num_layers=1,hidden_size=100,linear_layers=0,**kwargs):
        super().__init__()
        self.rnn = RNN(input_size=input_size,hidden_size=hidden_size,num_layers=num_layers,**kwargs)
        self.final = SeqLinear(hidden_size,output_size,hidden_size=hidden_size,hidden_layer=linear_layers,act=nn.LeakyReLU)
    def forward(self, x, h_init=None):
        out,_ = self.rnn(x,h_init)
        out = self.final(out)
        return out

In [ ]:
model = SimpleRNN(2,1,2,stateful=False,normalization='batchnorm')
lrn = Learner(db,model,loss_func=nn.MSELoss())#.fit(10)

In [ ]:
model = SimpleRNN(2,1,2,stateful=True,rnn_type='lstm')
lrn = Learner(db,model,loss_func=nn.MSELoss()).fit(1)

epoch,train_loss,valid_loss,time
0,12.545322,10.853215,00:03
1,9.312776,0.379804,00:03
2,5.893811,0.504065,00:03
3,4.133477,0.198409,00:03
4,3.085494,0.205462,00:03
5,2.382248,0.135016,00:03
6,1.890279,0.116953,00:03
7,1.525577,0.096417,00:03
8,1.245772,0.084401,00:03
9,1.027150,0.067469,00:03


In [ ]:
# model = SimpleRNN(2,1,2,stateful=True,rnn_type='sru')
# lrn = Learner(db,model,loss_func=nn.MSELoss()).fit(1)

epoch,train_loss,valid_loss,time
0,11.825572,9.369892,00:02
1,9.710959,5.947149,00:02
2,7.971813,3.963869,00:02
3,6.701194,3.241173,00:02
4,5.775784,2.727892,00:02
5,5.022538,2.290882,00:02
6,4.410403,1.982903,00:02
7,3.888186,1.754528,00:02
8,3.451751,1.591574,00:02
9,3.070811,1.448020,00:02


### Residual RNN

In [ ]:
#export
class ResidualBlock_RNN(nn.Module):
    
    @delegates(RNN, keep=True)
    def __init__(self, input_size,hidden_size,**kwargs):
        super().__init__()
        self.rnn1 = RNN(input_size,hidden_size,num_layers=1,**kwargs)
        self.rnn2 = RNN(hidden_size,hidden_size,num_layers=1,**kwargs)
        self.residual = SeqLinear(input_size,hidden_size,hidden_layer=0) if hidden_size!=input_size else noop

    def forward(self, x):
        out,_ = self.rnn1(x)
        out,_ = self.rnn2(out)
        return out+self.residual(x)

In [ ]:
#export
class SimpleResidualRNN(nn.Sequential):
    
    @delegates(ResidualBlock_RNN, keep=True)
    def __init__(self, input_size,output_size,num_blocks=1,hidden_size=100,**kwargs):
        super().__init__()
        for i in range(num_blocks):
            self.add_module('rnn%d'%i, ResidualBlock_RNN(input_size if i==0 else hidden_size,hidden_size,**kwargs))

        self.add_module('linear', SeqLinear(hidden_size,output_size,hidden_size,hidden_layer=1))

In [ ]:
model = SimpleResidualRNN(2,1,1,stateful=False,normalization='')
lrn = Learner(db,model,loss_func=nn.MSELoss()).fit(1)

epoch,train_loss,valid_loss,time
0,10.496976,6.018875,00:03


### Dense RNN

In [ ]:
#export
class DenseLayer_RNN(nn.Module):
    
    @delegates(RNN, keep=True)
    def __init__(self, input_size,hidden_size,**kwargs):
        super().__init__()
        self.rnn1 = RNN(input_size,hidden_size,num_layers=1,**kwargs)
        self.rnn2 = RNN(hidden_size,hidden_size,num_layers=1,**kwargs)

    def forward(self, x):
        out,_ = self.rnn1(x)
        out,_ = self.rnn2(out)
        return torch.cat([x, out], 2) 
    
class DenseBlock_RNN(nn.Sequential):
    @delegates(DenseLayer_RNN, keep=True)
    def __init__(self, num_layers, num_input_features,growth_rate,**kwargs):
        super().__init__()
        for i in range(num_layers):
            self.add_module('denselayer%d' % i, DenseLayer_RNN(num_input_features + i * growth_rate,growth_rate,**kwargs))
            

In [ ]:
#export
class DenseNet_RNN(nn.Sequential):
    
    @delegates(RNN, keep=True)
    def __init__(self, input_size, output_size, growth_rate=32, block_config=(3,3),
                 num_init_features=32,**kwargs):

        super().__init__()
        self.add_module('rnn0', Sequential_RNN(input_size, num_init_features,1,**kwargs))      

        # Each denseblock
        num_features = num_init_features  
        for i, num_layers in enumerate(block_config):
            self.add_module('denseblock%d' % i, DenseBlock_RNN(num_layers, num_features,growth_rate=growth_rate,**kwargs))
            num_features=num_features + num_layers * growth_rate
            
            self.add_module('transition%d' % i,  Sequential_RNN(num_features, num_features // 2,1,**kwargs))
            num_features = num_features // 2
        self.add_module('final',  SeqLinear(num_features, output_size ,hidden_layer=0))

In [ ]:
model = DenseNet_RNN(2,1,growth_rate=10,block_config=(1,1),num_init_features=2)
lrn = Learner(db,model,loss_func=nn.MSELoss()).fit(1)

### Seperate RNN

In [ ]:
#export
class SeperateRNN(nn.Module):
    
    @delegates(RNN, keep=True)
    def __init__(self,input_list,output_size,num_layers=1,hidden_size=100,linear_layers=1,**kwargs):
        super().__init__()
        self.input_list = input_list
        
        rnn_width = hidden_size//len(input_list)
        
        self.rnns =nn.ModuleList([ RNN(input_size=len(n),hidden_size=rnn_width,num_layers=1,**kwargs)
                        for n in input_list])
        
        self.rnn =RNN(input_size=rnn_width*len(input_list),hidden_size=hidden_size,
                      num_layers=num_layers,**kwargs)
        self.final = SeqLinear(hidden_size,output_size,hidden_size=hidden_size,hidden_layer=linear_layers)

    def forward(self, x):      
        rnn_out = [rnn(x[...,group])[0] for rnn,group in zip(self.rnns,self.input_list)]
        out = torch.cat(rnn_out,dim=-1)
        out,_ = self.rnn(out)
        out = self.final(out)
        return out

In [ ]:
model = SeperateRNN([[0],[1],[0,1]],1,1,stateful=False,rnn_type='lstm')
lrn = Learner(db,model,loss_func=nn.MSELoss()).fit(1)

epoch,train_loss,valid_loss,time
0,12.840867,11.523138,00:04


## CNNs

In [ ]:
#export
class CausalConv1d(torch.nn.Conv1d):
    def __init__(self,
                 in_channels,
                 out_channels,
                 kernel_size,
                 stride=1,
                 dilation=1,
                 groups=1,
                 bias=True,
                stateful=False):

        super().__init__(
            in_channels,
            out_channels,
            kernel_size=kernel_size,
            stride=stride,
            padding=0,
            dilation=dilation,
            groups=groups,
            bias=bias)
        self.__init_size = (kernel_size - 1) * dilation
        self.x_init = None
        self.stateful = stateful
        
    def forward(self, x):
        if self.x_init is not None and self.x_init.shape[0] != x.shape[0]: 
            self.x_init = None
        
        if self.x_init is None or not self.stateful:
            self.x_init = torch.zeros((x.shape[0],x.shape[1],self.__init_size),device=x.device)
            
        x = torch.cat([self.x_init,x],dim=-1)
        
        out = super().forward(x)
    
        if self.stateful: self.x_init =to_detach( x[...,-self.__init_size:], cpu=False, gather=False) 
            
        return out
    
    def reset_state(self):
        self.x_init = None

In [ ]:
#export
@delegates(CausalConv1d, keep=True)
def CConv1D(input_size,output_size,kernel_size=2,activation = Mish,wn=True, bn = False, **kwargs):
    conv = CausalConv1d(input_size,output_size,kernel_size,**kwargs)
    if wn: conv = weight_norm(conv)
    act = activation() if activation is not None else None
    bn = nn.BatchNorm1d(input_size) if bn else None
    m = [m for m in [bn,conv,act] if m is not None]
    return nn.Sequential(*m)

In [ ]:
#export
@delegates(CausalConv1d, keep=True)
class TCN_Block(nn.Module):
    def __init__(self,input_size,output_size,num_layers=1,
                 activation = Mish,wn=True, bn = False,stateful=False, **kwargs):
        super().__init__()
        
        layers=[]
        for _ in range(num_layers):
            conv = CausalConv1d(input_size,output_size,2,stateful=stateful,**kwargs)
            if wn: conv = weight_norm(conv)
            act = activation() if activation is not None else None
            bn = nn.BatchNorm1d(input_size) if bn else None
            layers += [m for m in [bn,conv,act] if m is not None]
            
        self.layers = nn.Sequential(*layers)
        
        self.residual = nn.Conv1d(input_size,output_size,kernel_size=1) if output_size!=input_size else None
        
    def forward(self, x):      
        out = self.layers(x)
        out = out + (x if self.residual is None else self.residual(x))  
        return out

In [ ]:
#export
class TCN(nn.Module):
    def __init__(self,input_size,output_size,hl_depth=1,hl_width=10,act = Mish,bn=False,stateful=False):
        super().__init__()
        
        conv_layers = [TCN_Block(input_size if i==0 else hl_width,hl_width,
                                      dilation=2**(i),bn=bn,activation=act,stateful=stateful)
                                          for i in range(hl_depth)]
        self.conv_layers = nn.Sequential(*conv_layers)
        self.final = nn.Conv1d(hl_width,output_size,kernel_size=1)
        
    def forward(self, x):      
        x_in = x.transpose(1,2)
        out = self.conv_layers(x_in)
        out = self.final(out).transpose(1,2)
        return out

In [ ]:
model = TCN(2,1,hl_depth=3)
lrn = Learner(db,model,loss_func=nn.MSELoss()).fit(1)

epoch,train_loss,valid_loss,time
0,15.326912,14.688392,00:02


In [ ]:
#export
class SeperateTCN(nn.Module):
    def __init__(self,input_list,output_size,hl_depth=1,hl_width=10,act = Mish,bn=False,stateful=False,final_layer=3):
        super().__init__()
        self.input_list =np.cumsum([0] + input_list)
        
        tcn_width = hl_width//len(input_list)
        layers = [ [TCN_Block(n if i==0 else tcn_width,tcn_width,
                                      dilation=2**(i),bn=bn,activation=act)
                                          for i in range(hl_depth)]
                        for n in input_list]
        self.layers = nn.ModuleList([nn.Sequential(*l) for l in layers])
        
        self.rec_field = (2**hl_depth)-1
        self.final = SeqLinear(tcn_width*len(input_list),output_size,hidden_size=hl_width,hidden_layer=final_layer)
        self.x_init = None
        self.stateful = stateful
        
    def forward(self, x):      
        if self.x_init is not None:
            if self.x_init.shape[0] != x.shape[0]: 
                self.x_init = None
            elif self.stateful: 
                x = torch.cat([self.x_init,x],dim=1)
        
        tcn_out = [layer(x[...,self.input_list[i]:self.input_list[i+1]].transpose(1,2)) 
                   for i,layer in enumerate(self.layers)]
        out = torch.cat(tcn_out,dim=1).transpose(1,2)
        
        out = self.final(out)
        
        if self.stateful:
            if self.x_init is not None: out = out[:,self.rec_field:]
            self.x_init = x[:,-self.rec_field:]
            
        return out
    
    def reset_state(self):
        self.x_init = None

## CRNNs

In [ ]:
#export
class CRNN(nn.Module):
    def __init__(self,input_size,output_size,num_ft=10,num_cnn_layers=4,num_rnn_layers=2,hs_cnn=10,hs_rnn=10,
         hidden_p=0, input_p=0, weight_p=0, rnn_type='gru',stateful=True):
        super().__init__()
        self.cnn = TCN(input_size,num_ft,num_cnn_layers,hs_cnn,act=nn.ReLU,stateful=stateful)
        self.rnn = SimpleRNN(num_ft,output_size,num_layers=num_rnn_layers,hidden_size=hs_rnn,
                       hidden_p=hidden_p, input_p=input_p, weight_p=weight_p, 
                        rnn_type=rnn_type,stateful=stateful)
        
    def forward(self, x):
        return self.rnn(self.cnn(x))
    

In [ ]:
model = CRNN(2,1,10)
lrn = Learner(db,model,loss_func=nn.MSELoss()).fit(1)

epoch,train_loss,valid_loss,time
0,9.231150,8.298079,00:02


In [ ]:
model = CRNN(2,1,10,rnn_type='gru')
lrn = Learner(db,model,loss_func=nn.MSELoss()).fit(1)

epoch,train_loss,valid_loss,time
0,14.739855,13.977013,00:02


In [ ]:
#export
class SeperateCRNN(nn.Module):
    def __init__(self,input_list,output_size,num_ft=10,num_cnn_layers=4,num_rnn_layers=2,hs_cnn=10,hs_rnn=10,
         hidden_p=0, input_p=0, weight_p=0, rnn_type='gru',stateful=True):
        super().__init__()
        self.cnn = SeperateTCN(input_list,num_ft,num_cnn_layers,hs_cnn,act=nn.ReLU,stateful=stateful,final_layer=0)
        self.rnn = SimpleRNN(num_ft,output_size,num_layers=num_rnn_layers,hidden_size=hs_rnn,
                       hidden_p=hidden_p, input_p=input_p, weight_p=weight_p, 
                        rnn_type=rnn_type,stateful=stateful)
        
    def forward(self, x):
        return self.rnn(self.cnn(x))

## Autoregressive Models

In [ ]:
#export
class Normalizer1D(nn.Module):
    _epsilon = 1e-16

    def __init__(self, mean, std):
        super(Normalizer1D, self).__init__()
        self.register_buffer('std', std.clone().detach() + self._epsilon)
        self.register_buffer('mean', mean.clone().detach())

    def normalize(self, x):
        if x.device != self.mean.device:
            self.mean = self.mean.to(x.device)
            self.std = self.std.to(x.device)
        return (x-self.mean)/self.std

    def unnormalize(self, x):
        if x.device != self.mean.device:
            self.mean = self.mean.to(x.device)
            self.std = self.std.to(x.device)
        return x*self.std + self.mean

In [ ]:
# #export
# class AR_Model(nn.Module):
#     def __init__(self,model,ar=True,receptive_field=1,hs=False,stateful=False):
#         super().__init__()
#         self.model = model
#         self.ar = ar
#         self.receptive_field = receptive_field #receptive field, sequence length to pad at the beginning
#         self.hs = hs 
#         self.norm = None
#         self.stateful = stateful
#         self.y_init = None
    
#     def init_normalize(self, batch,axes = [0,1]):
#         x = batch[1]
#         mean = x.mean(axes, keepdim=True)
#         std = x.std(axes, keepdim=True)
#         self.norm = Normalizer1D(mean,std)
        
#     def init_normalize_values(self, mean, std):
#         self.norm = Normalizer1D(mean,std)
        
#     def forward(self, u,y):
        
#         if self.stateful and self.y_init is not None and self.y_init.shape[0] == y.shape[0]:
#             y_in = torch.cat([self.y_init,y[:,:-1]],dim=1)
#         else:
#             y_in = F.pad(y[:,:-1,:],[0,0,1,0])#first value is always zero, needs to be stored for tbptt
        
#         if self.ar: #autoregressive mode
#             y_e = y_in
#             hs = None
#             for i in range(y_e.shape[1]):
#                 if i < self.receptive_field:
#                     y_in = F.pad(y_in[:, :i+1], [0,0,self.receptive_field-i-1, 0])
#                     u_in = F.pad(u[:, :i+1], [0,0,self.receptive_field-i-1, 0])
#                 else:
#                     y_in = y_e[:, i-self.receptive_field:i]
#                     u_in = u[:, i-self.receptive_field+1:i+1]
                    
#                 if self.norm is not None: y_in=self.norm.normalize(y_in)
                    
#                 x = torch.cat((u_in, y_in), 2)
                
#                 if self.hs: #model has a hidden state as input
#                     y_next,hs = self.model(x,hs)
#                 else:
#                     y_next = self.model(x)
#                 y_e[:, i] = y_next[:, -1]
#         else: #teacherforcing mode

#             if self.norm is not None: y_in=self.norm.normalize(y_in)
            
#             x = torch.cat([u,y_in],dim=2)
#             if self.hs: #model has a hidden state as output
#                 y_e,_ = self.model(x)
#             else:
#                 y_e = self.model(x)
                
#         if self.stateful:
#             self.y_init = to_detach(y_e[:,-1:], cpu=False, gather=False) 
#         return y_e
    
#     def reset_state(self):
#         self.y_init = None

In [ ]:
#export
class AR_Model(nn.Module):
    def __init__(self,model,ar=True,stateful=False):
        super().__init__()
        self.model = model
        self.ar = ar
        self.norm = None
        self.stateful = stateful
        self.y_init = None
    
    def init_normalize(self, batch,axes = [0,1]):
        x = batch[1]
        mean = x.mean(axes, keepdim=True)
        std = x.std(axes, keepdim=True)
        self.norm = Normalizer1D(mean,std)
        
    def init_normalize_values(self, mean, std):
        self.norm = Normalizer1D(mean,std)
        
    def forward(self, u,y):
        
        if self.stateful and self.y_init is not None and self.y_init.shape[0] == u.shape[0]:
            y_in = torch.cat([self.y_init,y[:,:-1]],dim=1)
        else:
            y_in = F.pad(y[:,:-1,:],[0,0,1,0])#first value is always zero, needs to be stored for tbptt
        
        if self.ar: #autoregressive mode
            y_e = []
            
            y_next = y_in[:,:1]
            
            for i in range(u.shape[1]):
                
                if self.norm is not None: y_next=self.norm.normalize(y_next)
                x = torch.cat((u[:, i:i+1], y_next), dim=2)
                y = self.model(x)
                    
                y_next = y[:, -1:]
                y_e.append(y_next)
                    
            y_e = torch.cat(y_e,dim=1)
            
        else: #teacherforcing mode

            if self.norm is not None: y_in=self.norm.normalize(y_in)
            
            x = torch.cat([u,y_in],dim=2)
            y_e = self.model(x)
                
        if self.stateful:
            self.y_init = to_detach(y_e[:,-1:], cpu=False, gather=False) 
            
#             import pdb; pdb.set_trace()
        return y_e
    
    def reset_state(self):
        self.y_init = None

In [ ]:
model = AR_Model(SimpleRNN(3,1),ar=True)
model.init_normalize(db.one_batch())

In [ ]:
#hide
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_models.ipynb.
Converted 01a_IndRNN.ipynb.
Converted 02_learner.ipynb.
Converted 03_dataloaders.ipynb.
Converted 11_dualrnn.ipynb.
Converted 12_TensorQuaternions.ipynb.
Converted 13_HPOpt.ipynb.
Converted index.ipynb.
